In [2]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator
from datetime import timedelta
from ipywidgets import interact, IntSlider
from IPython.display import display
import sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
# import seaborn as sns
# from matplotlib import pyplot as plt
# %matplotlib inline
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler
from imblearn.under_sampling import RandomUnderSampler

# from pycaret.classification import * 
from pycaret.regression import * 

# pd.options.display.max_columns = 5000
# pd.options.display.max_rows = 5000

injured_pred = 'injury_duration'

# simple_features = ['Height', 'Weight', 'age','cum_injury_total', 'weeks_since_last_injury', 'Min_cum','Serie A_cum',
#  'Premier League_cum', 'La Liga_cum', 'Ligue 1_cum', 'Bundesliga_cum', 'Champions Lg_cum', 'Europa Lg_cum', 'FIFA World Cup_cum', 'UEFA Nations League_cum', 'UEFA Euro_cum',
#  'Copa América_cum', 'Away_cum', 'Home_cum', 'Neutral_cum']

In [3]:
# Load Dataset

# Renzo's Path
# dataset = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/complete_final_df_5.csv')
#
# Sravan's Path
# dataset = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/dataset_for_model_final.csv')

# Sravan's Parquet Path
dataset = pd.read_parquet('dataframes_blog/dataset_for_model_final.parquet') 

In [4]:
injury_duration_df = dataset[['FBRefID', 'cum_injury_total', 'Injury','injury_count']].groupby(['FBRefID', 'cum_injury_total', 'Injury']).sum().reset_index()
injury_duration_df = injury_duration_df[injury_duration_df['Injury'] != '0'].rename(columns={'injury_count':'injury_duration'})
new_player_df_1 = pd.merge(dataset, injury_duration_df, left_on=['FBRefID', 'cum_injury_total', 'Injury'], right_on=['FBRefID', 'cum_injury_total', 'Injury'], how='left')
new_player_df_1['injury_duration'] = new_player_df_1['injury_duration'].fillna(0)

In [5]:
dataset_injury = new_player_df_1[new_player_df_1['injury_duration']!=0]

In [6]:
dataset_injury = dataset_injury.drop_duplicates(['Injury', 'injury_week', 'injury_year'])
dataset_injury = dataset_injury[dataset_injury['Injury']!='Unknown Injury'].reset_index(drop=True)

In [7]:
feature_list = list(dict(dataset_injury['Injury'].value_counts()).keys())

for col in feature_list:
    dataset_injury.loc[dataset_injury['Injury'] == col, col] = 1
    dataset_injury.loc[dataset_injury['Injury'] != col, col] = 0

In [8]:
extended_features = ['Height', 'Weight', 'cum_week', 'defender', 'attacker', 'midfielder', 'goalkeeper', 'right_foot', 'cum_injury_total',
 'weeks_since_last_injury', 'Min_cum', 'Gls_cum', 'Ast_cum', 'PK_cum', 'PKatt_cum', 'Sh_cum', 'SoT_cum', 'CrdY_cum',
 'CrdR_cum', 'Touches_cum', 'Press_cum', 'Tkl_cum', 'Int_cum', 'Blocks_cum', 'xG_cum', 'npxG_cum', 'xA_cum',
 'SCA_cum', 'GCA_cum', 'Cmp_cum', 'Att_cum', 'Prog_cum', 'Carries_cum', 'Prog.1_cum', 'Succ_cum', 'Att.1_cum',
 'Fls_cum', 'Fld_cum', 'Off_cum', 'Crs_cum', 'TklW_cum', 'OG_cum', 'PKwon_cum', 'PKcon_cum', 'Serie A_cum', 'Premier League_cum',
 'La Liga_cum', 'Ligue 1_cum', 'Bundesliga_cum', 'Champions Lg_cum', 'Europa Lg_cum', 'FIFA World Cup_cum',
 'UEFA Nations League_cum', 'UEFA Euro_cum', 'Copa América_cum', 'Away_cum', 'Home_cum', 'Neutral_cum',
                     'Injury_Recurrence_cum'] + list(dict(dataset_injury['Injury'].value_counts()).keys())[:30]

In [9]:
injury_recurrence = pd.DataFrame(dataset_injury.groupby(['FBRefID','Injury', 'injury_week', 'injury_year'])['Injury'].count()).rename(columns={'Injury':'Injury_Recurrence'}).reset_index().sort_values(by=['FBRefID','injury_year', 'injury_week'])
injury_recurrence['Injury_Recurrence_cum'] = injury_recurrence.groupby(['FBRefID','Injury'])['Injury_Recurrence'].cumsum()


In [10]:
injury_recurrence

,FBRefID,Injury,injury_week,injury_year,Injury_Recurrence,Injury_Recurrence_cum
4,0000acda,Calf Problems,46.0,2013.0,1,1
8,0000acda,Thigh Problems,49.0,2014.0,1,1
0,0000acda,Adductor problems,47.0,2015.0,1,1
7,0000acda,Muscle Fatigue,15.0,2016.0,1,1
6,0000acda,Distortion of the ankle,31.0,2016.0,1,1
...,...,...,...,...,...,...
17275,ffbbc83b,Abdominal Strain,51.0,2020.0,1,1
17286,ffec9769,Meniscal Injury,43.0,2012.0,1,1
17284,ffec9769,Bruise,4.0,2014.0,1,1
17285,ffec9769,Leg Injury,17.0,2015.0,1,1


In [11]:
dataset_injury_1 = pd.merge(dataset_injury, injury_recurrence, on=['FBRefID', 'Injury', 'injury_week', 'injury_year'], how='left')


In [16]:
exp_reg = setup(dataset_injury_1[extended_features + [injured_pred]], target = injured_pred, fold=10, feature_selection=True)


,Description,Value
0,session_id,7548
1,Target,injury_duration
2,Original Data,"(17288, 90)"
3,Missing Values,False
4,Numeric Features,40
5,Categorical Features,49
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(12101, 96)"


In [17]:
best_model = compare_models(turbo=False)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,3.5997,35.6456,5.9610,0.1884,0.7785,1.6533,0.1520
ridge,Ridge Regression,3.5994,35.6753,5.9636,0.1877,0.7791,1.6530,0.1490
lr,Linear Regression,3.6013,35.7031,5.9659,0.1870,0.7797,1.6545,0.2780
gbr,Gradient Boosting Regressor,3.5818,35.7226,5.9673,0.1869,0.7675,1.6325,0.1840
ard,Automatic Relevance Determination,3.5923,35.7161,5.9668,0.1868,0.7771,1.6481,0.2020
lightgbm,Light Gradient Boosting Machine,3.5875,36.2305,6.0093,0.1755,0.7728,1.6186,0.0410
omp,Orthogonal Matching Pursuit,3.6358,36.2886,6.0147,0.1738,0.7846,1.6759,0.1470
tr,TheilSen Regressor,3.6707,36.4376,6.0268,0.1706,0.7826,1.6783,2.1820
mlp,MLP Regressor,3.7985,38.1746,6.1635,0.1329,0.8245,1.6861,0.8070
kr,Kernel Ridge,3.8224,40.2146,6.3275,0.0860,0.8163,1.6620,70.9830


In [ ]:
# df = pull()
# df.to_csv('results_1_year.csv', index=False)
df = pull().sort_values(by=['F1'], ascending=False)

In [10]:
df

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.7520,0.6318,0.3903,0.1207,0.1843,0.0839,0.1041,79.340
gbc,Gradient Boosting Classifier,0.8345,0.6419,0.2455,0.1367,0.1756,0.0919,0.0967,302.775
lr,Logistic Regression,0.6738,0.5990,0.4502,0.1013,0.1654,0.0546,0.0783,122.505
ridge,Ridge Classifier,0.8370,0.0000,0.2233,0.1301,0.1645,0.0811,0.0847,4.340
lda,Linear Discriminant Analysis,0.8370,0.6136,0.2233,0.1301,0.1644,0.0811,0.0847,8.845
nb,Naive Bayes,0.4668,0.5831,0.6314,0.0874,0.1506,0.0293,0.0477,4.845
knn,K Neighbors Classifier,0.7232,0.5384,0.2958,0.0859,0.1331,0.0245,0.0312,509.850
svm,SVM - Linear Kernel,0.5622,0.0000,0.4735,0.1168,0.1173,0.0311,0.0457,96.580
dt,Decision Tree Classifier,0.7823,0.5104,0.1951,0.0806,0.1140,0.0138,0.0156,20.330
qda,Quadratic Discriminant Analysis,0.6751,0.5681,0.3238,0.0830,0.0917,0.0100,0.0172,7.375


In [12]:
# Select best model
model = create_model(df.index[0], fold=5)
save_model(model, 'model_1_year')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7359,0.6265,0.4045,0.1160,0.1803,0.0773,0.0990
1,0.7256,0.6360,0.4322,0.1173,0.1845,0.0806,0.1056
2,0.7545,0.6208,0.3624,0.1153,0.1749,0.0740,0.0909
3,0.7362,0.6308,0.4039,0.1160,0.1802,0.0773,0.0988
4,0.7345,0.6389,0.4006,0.1145,0.1781,0.0748,0.0958
Mean,0.7373,0.6306,0.4007,0.1158,0.1796,0.0768,0.0980
SD,0.0094,0.0065,0.0223,0.0009,0.0031,0.0023,0.0048


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[],
                                       target='injured_in_26_week',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 n...
                  Advanced_Feature_Selection_Classic(ml_usecase='classification',
                                                     n_jobs=-1,
                                                     random_state=8301,
                 

In [14]:
tuned_model = tune_model(model, optimize = 'F1')
save_model(tuned_model, 'model_1_year_tuned')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7363,0.6396,0.4127,0.1180,0.1835,0.0809,0.1036
1,0.7292,0.6422,0.4297,0.1183,0.1856,0.0822,0.1070
Mean,0.7327,0.6409,0.4212,0.1182,0.1845,0.0815,0.1053
SD,0.0036,0.0013,0.0085,0.0002,0.0010,0.0007,0.0017


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[],
                                       target='injured_in_26_week',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 n...
                  Advanced_Feature_Selection_Classic(ml_usecase='classification',
                                                     n_jobs=-1,
                                                     random_state=8301,
                 

In [6]:
# Loading the saved model
model_1_year_tuned = load_model('model_1_year_tuned')

Transformation Pipeline and Model Successfully Loaded


In [7]:
model_1_year_tuned

Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(ml_usecase='classification',
                                      target='injured_in_52_week')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         'fr...
                ('fix_perfect', Remove_100(target='injured_in_52_week')),
                ('clean_names', Clean_Colum_Names()),
                ('feature_select',
                 Advanced_Feature_Selection_Classic(n_jobs=-1,
                                                    random_state=2889,
               